# The battle of Neightborhoods

Find a potential location to start a korean restaurant in Kwun Tong District, Hong Kong

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # plotting library

     |████████████████████████████████| 112kB 6.1MB/s eta 0:00:01


In [2]:
CLIENT_ID = '40O3R50PSWFMNGWGHJQKVSNJ2GLMLLWYJ43NNXS1HU5XL1VQ' # your Foursquare ID
CLIENT_SECRET = 'HILPUJA0ZVVNID4CK0ZWVJS010I5XFUWLWM5S3WTGO3VTPYS' # your Foursquare Secret
ACCESS_TOKEN = 'LA5ATAVFXHJM2K3NU2IRSQUKGDNVMIS0SC2C1XNIMG2F1R5W' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
categoryid = '4bf58dd8d48988d113941735'
radius = 5000

In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
address = ['Central and Western','Eastern','Southern','Wan Chai','Sham Shui Po','Wong Tai Sin','Kowloon City','Yau Tsim Mong','Islands','Kwai Tsing','North','Sai Kung','Sha Tin','Tai Po','Tsuen Wan','Tuen Mun','Yuen Long']

for i,add in enumerate(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(add+' District, Hong Kong')
    latitude = location.latitude
    longitude = location.longitude

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, categoryid, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    if i==0:
        main_df=dataframe
    else:
        main_df=pd.concat([main_df,dataframe])

filtered_columns = ['name', 'categories'] + [col for col in main_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = main_df.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Won Pung Won (元豐園),Korean Restaurant,22-24 Prat Ave,22.298585,114.175014,"[{'label': 'display', 'lat': 22.2985851866778,...",3783,HK,尖沙咀,Kowloon City,香港,"[22-24 Prat Ave, Kowloon City, 香港]",NaN,NaN,4b4bb83bf964a5202da526e3
1,Goobne Chicken,Wings Joint,"25/F, 18 Tang Lung Street",22.279113,114.181567,"[{'label': 'display', 'lat': 22.27911262235827...",3415,HK,铜锣湾,NaN,香港,"[25/F, 18 Tang Lung Street, 香港]",NaN,NaN,57d56fac498eda3cf4991a55
2,Seoul Recipe,Korean Restaurant,7 Staunton Street,22.281888,114.153069,"[{'label': 'display', 'lat': 22.281888, 'lng':...",902,HK,中環,中西區,香港,"[7 Staunton Street, 中西區, 香港]",NaN,NaN,5f795082ae7ab8403476d8fc
3,韓潮,Korean Restaurant,21 Wharf Road,22.292136,114.195280,"[{'label': 'display', 'lat': 22.292136, 'lng':...",5166,HK,北角,NaN,香港,"[21 Wharf Road, 香港]",NaN,NaN,604482c74679bb39ca14fb50
4,韓Cook BBQ,Korean Restaurant,"G/F, Shop F1, 27E Tak Man St",22.306343,114.188305,"[{'label': 'display', 'lat': 22.30634271482674...",5376,HK,Hung Hom,Kowloon,香港,"[G/F, Shop F1, 27E Tak Man St, Kowloon, 香港]",NaN,NaN,53ef3757498efa3285eaa576


In [5]:
dataframe_filtered=dataframe_filtered[dataframe_filtered['city']!='观塘']
dataframe_filtered=dataframe_filtered[~((dataframe_filtered['address']!=np.NaN) & (dataframe_filtered['address'].str.contains('Kwun Tong')))]

dataframe_filtered=dataframe_filtered.drop_duplicates(subset=['lat','lng','name'])

In [6]:
add='Kowloon,Hong Kong'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(add)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

from folium.plugins import FastMarkerCluster
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)
venues_map.add_child(FastMarkerCluster(dataframe_filtered[['lat', 'lng']].values.tolist()))
venues_map.save("save_file.html")

venues_map

22.3116452 114.1767885


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
#Potential location in Kwun Tong District
potential_loc=pd.DataFrame({'Name':['APM','Megabox','Kwun Tong Promenade','Lei Yue Mun'],'Lat':[22.3075204366,22.319802,22.309587,22.2884368],'Lng':[114.221699113,114.20815,114.219262,114.2377771]})
potential_loc

potential_venues = getNearbyVenues(names=potential_loc['Name'],
                                   latitudes=potential_loc['Lat'],
                                   longitudes=potential_loc['Lng']
                                  )

potential_venues.head()

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,APM,22.30752,114.221699,Kwun Tong Promenade (觀塘海濱花園),22.310726,114.218341,Park
1,APM,22.30752,114.221699,My Burger,22.310905,114.226265,Burger Joint
2,APM,22.30752,114.221699,Osage Gallery,22.308510,114.225360,Art Gallery
3,APM,22.30752,114.221699,Apple apm Hong Kong,22.311976,114.224943,Electronics Store
4,APM,22.30752,114.221699,Red Tea Café (紅茶冰室),22.311921,114.222548,Cha Chaan Teng


In [9]:
kr_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

print(kr_venues.shape)
kr_venues.head()

(15585, 7)


,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Won Pung Won (元豐園),22.298585,114.175014,Hyatt Regency Hong Kong Tsim Sha Tsui (香港尖沙咀凱悅酒店),22.297452,114.173917,Hotel
1,Won Pung Won (元豐園),22.298585,114.175014,Kowloon Shangri-La (九龍香格里拉大酒店),22.297371,114.176921,Hotel
2,Won Pung Won (元豐園),22.298585,114.175014,Sangeetha,22.296518,114.176478,South Indian Restaurant
3,Won Pung Won (元豐園),22.298585,114.175014,Empire International Tailors,22.297744,114.177171,Tailor Shop
4,Won Pung Won (元豐園),22.298585,114.175014,Japanese Restaurant Doraya (定食Doraya),22.299643,114.174290,Japanese Restaurant


In [10]:
merge_venues=pd.concat([kr_venues,potential_venues])

In [11]:
# one hot encoding
merge_onehot = pd.get_dummies(merge_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
merge_onehot['Name'] = merge_venues['Name'] 

# move neighborhood column to the first column
fixed_columns = [merge_onehot.columns[-1]] + list(merge_onehot.columns[:-1])
merge_onehot =merge_onehot[fixed_columns]

merge_grouped = merge_onehot.groupby('Name').mean().reset_index()

potential_grouped=merge_grouped[(merge_grouped['Name']=='APM')|(merge_grouped['Name']=='Megabox')|(merge_grouped['Name']=='Kwun Tong Promenade')|(merge_grouped['Name']=='Lei Yue Mun')]
kr_grouped=merge_grouped[~((merge_grouped['Name']=='APM')|(merge_grouped['Name']=='Megabox')|(merge_grouped['Name']=='Kwun Tong Promenade')|(merge_grouped['Name']=='Lei Yue Mun'))]

In [12]:
potential_grouped.reset_index(inplace=True,drop=True)
potential_grouped

,Name,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo
0,APM,0.011765,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Kwun Tong Promenade,0.011364,0.0,0.0,0.011364,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Lei Yue Mun,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Megabox,0.011628,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.011628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
kr_grouped.reset_index(inplace=True,drop=True)
kr_grouped.head()

,Name,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo
0,119 Korea Station,0.016949,0.00,0.0,0.0,0.0,0.00,0.016949,0.0,0.016949,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0
1,Apgujeong Tent Bar (狎鷗亭居酒屋),0.000000,0.00,0.0,0.0,0.0,0.01,0.000000,0.0,0.000000,...,0.01,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0
2,Arirang Korean Restaurant (阿里郎韓國餐廳),0.000000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,...,0.00,0.01,0.0,0.0,0.0,0.0,0.01,0.000000,0.01,0.0
3,Asahikawa Japanese Restaurant (旭川日本料理),0.000000,0.01,0.0,0.0,0.0,0.00,0.010000,0.0,0.000000,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0
4,Azooms,0.000000,0.00,0.0,0.0,0.0,0.00,0.011628,0.0,0.011628,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.011628,0.00,0.0


In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Name'] = kr_grouped['Name']

for ind in np.arange(kr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,119 Korea Station,Chinese Restaurant,Noodle House,Dessert Shop,Fast Food Restaurant,Shopping Mall,Japanese Restaurant,Coffee Shop,Dim Sum Restaurant,Train Station,Market
1,Apgujeong Tent Bar (狎鷗亭居酒屋),Hotel,Café,Japanese Restaurant,Hotel Bar,Chinese Restaurant,Korean Restaurant,Shopping Mall,Coffee Shop,Buffet,Sushi Restaurant
2,Arirang Korean Restaurant (阿里郎韓國餐廳),Hotel,Coffee Shop,Café,Japanese Restaurant,Bakery,Thai Restaurant,Hong Kong Restaurant,Italian Restaurant,Noodle House,Cantonese Restaurant
3,Asahikawa Japanese Restaurant (旭川日本料理),Chinese Restaurant,Noodle House,Japanese Restaurant,Café,Coffee Shop,Department Store,Pizza Place,Park,Thai Restaurant,Cantonese Restaurant
4,Azooms,Shopping Mall,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Noodle House,Cantonese Restaurant,Park,Dessert Shop,Cha Chaan Teng,Café


In [16]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

kr_grouped_clustering = kr_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

try:
    neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop('Cluster Labels',1)
except:
    print('nothing to drop')

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kr_merged = dataframe_filtered

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
kr_merged = kr_merged.join(neighborhoods_venues_sorted.set_index('Name'), on='name')

kr_merged # check the last columns!

nothing to drop


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Won Pung Won (元豐園),Korean Restaurant,22-24 Prat Ave,22.298585,114.175014,"[{'label': 'display', 'lat': 22.2985851866778,...",3783,HK,尖沙咀,Kowloon City,...,Hotel,Japanese Restaurant,Café,Hotel Bar,Steakhouse,Shopping Mall,Chinese Restaurant,Korean Restaurant,Coffee Shop,Buffet
1,Goobne Chicken,Wings Joint,"25/F, 18 Tang Lung Street",22.279113,114.181567,"[{'label': 'display', 'lat': 22.27911262235827...",3415,HK,铜锣湾,NaN,...,Chinese Restaurant,Hotel,Shopping Mall,Coffee Shop,Japanese Restaurant,Café,Fast Food Restaurant,Thai Restaurant,Cha Chaan Teng,Noodle House
2,Seoul Recipe,Korean Restaurant,7 Staunton Street,22.281888,114.153069,"[{'label': 'display', 'lat': 22.281888, 'lng':...",902,HK,中環,中西區,...,Japanese Restaurant,Bar,French Restaurant,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Yoga Studio,Wine Bar,Restaurant
3,韓潮,Korean Restaurant,21 Wharf Road,22.292136,114.195280,"[{'label': 'display', 'lat': 22.292136, 'lng':...",5166,HK,北角,NaN,...,Coffee Shop,Hotel,Hong Kong Restaurant,Noodle House,Sushi Restaurant,Cantonese Restaurant,Chinese Restaurant,Dessert Shop,Bakery,Burger Joint
4,韓Cook BBQ,Korean Restaurant,"G/F, Shop F1, 27E Tak Man St",22.306343,114.188305,"[{'label': 'display', 'lat': 22.30634271482674...",5376,HK,Hung Hom,Kowloon,...,Coffee Shop,Fast Food Restaurant,Hotel,Japanese Restaurant,Chinese Restaurant,Hotpot Restaurant,Dessert Shop,Café,Thai Restaurant,Snack Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,Gong's korean food,Korean Restaurant,NaN,22.441593,114.034367,"[{'label': 'display', 'lat': 22.44159313193514...",2188,HK,NaN,NaN,...,Chinese Restaurant,Noodle House,Fast Food Restaurant,Dessert Shop,Dim Sum Restaurant,Shopping Mall,Bakery,Sushi Restaurant,Market,Thai Restaurant
21,Onigiri 我呢家,Korean Restaurant,"Shop 12, G/F, Yee Fung Building, 31-35A Yau Su...",22.443352,114.031608,"[{'label': 'display', 'lat': 22.443352446032, ...",1858,HK,Yuen Long,Yuen Long District,...,Chinese Restaurant,Noodle House,Dessert Shop,Fast Food Restaurant,Shopping Mall,Coffee Shop,Dim Sum Restaurant,Bakery,Thai Restaurant,Café
22,Seoul Good,Korean Restaurant,"Shop A1, G/F, Fung Nin Bldg, 26-36 Tai Pei Tau...",22.444090,114.024425,"[{'label': 'display', 'lat': 22.44408983086645...",1452,HK,NaN,NaN,...,Chinese Restaurant,Noodle House,Dessert Shop,Shopping Mall,Fast Food Restaurant,Japanese Restaurant,Market,Asian Restaurant,Coffee Shop,Dim Sum Restaurant
23,景福宮,Korean Restaurant,NaN,22.442203,114.031173,"[{'label': 'display', 'lat': 22.442203, 'lng':...",1936,HK,NaN,NaN,...,Chinese Restaurant,Noodle House,Dessert Shop,Fast Food Restaurant,Shopping Mall,Café,Coffee Shop,Dim Sum Restaurant,Bakery,Thai Restaurant


In [42]:
query=pd.DataFrame(kr_merged[['Cluster Labels']].value_counts())
query.columns=['Number of Restaurant']
query

,Number of Restaurant
Cluster Labels,
3,46
1,42
7,27
4,26
9,16
2,14
6,12
5,8
0,8


In [18]:
potential_grouped_clustering = potential_grouped.drop('Name', 1)
potential_cluster=kmeans.predict(potential_grouped_clustering)
potential_cluster

array([3, 3, 1, 3], dtype=int32)

In [26]:
query1=kr_merged.loc[kr_merged['Cluster Labels'] == 3, kr_merged.columns[[0] + list(range(15, kr_merged.shape[1]-7))]]
query1.head()

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
24,Hanbros Korean Bulgogi (韓兄弟燒烤),3,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant
38,韓日燒自助餐廳,3,Noodle House,Chinese Restaurant,Hong Kong Restaurant
15,MoMoKu (韓樂),3,Fast Food Restaurant,Chinese Restaurant,Shopping Mall
28,MeokBang Korean BBQ & BAR (炑八韓烤),3,Fast Food Restaurant,Chinese Restaurant,Shopping Mall
24,Hansarang (韓盛朗),3,Coffee Shop,Chinese Restaurant,Cha Chaan Teng


In [30]:
print(query1[['name','1st Most Common Venue']].groupby('1st Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query1[['name','2nd Most Common Venue']].groupby('2nd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query1[['name','3rd Most Common Venue']].groupby('3rd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')

                       name
1st Most Common Venue      
Shopping Mall            14
Chinese Restaurant       13
Fast Food Restaurant     13
Thai Restaurant           3
Cha Chaan Teng            1
Coffee Shop               1
Noodle House              1
---------
                       name
2nd Most Common Venue      
Chinese Restaurant       18
Fast Food Restaurant     15
Shopping Mall             9
Dessert Shop              2
Cha Chaan Teng            1
Coffee Shop               1
---------
                       name
3rd Most Common Venue      
Fast Food Restaurant     13
Chinese Restaurant       12
Shopping Mall            10
Café                      2
Cha Chaan Teng            2
Coffee Shop               2
Hong Kong Restaurant      2
Noodle House              2
Japanese Restaurant       1
---------


In [31]:
query2=kr_merged.loc[kr_merged['Cluster Labels'] == 1, kr_merged.columns[[0] + list(range(15, kr_merged.shape[1]-7))]]
query2.head()

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Goobne Chicken,1,Chinese Restaurant,Hotel,Shopping Mall
3,韓潮,1,Coffee Shop,Hotel,Hong Kong Restaurant
4,韓Cook BBQ,1,Coffee Shop,Fast Food Restaurant,Hotel
6,The Charcoal Room,1,Coffee Shop,Cha Chaan Teng,Noodle House
10,Kim's Spoon by Baab,1,Coffee Shop,Chinese Restaurant,Park


In [32]:
print(query2[['name','1st Most Common Venue']].groupby('1st Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query2[['name','2nd Most Common Venue']].groupby('2nd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query2[['name','3rd Most Common Venue']].groupby('3rd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')

                       name
1st Most Common Venue      
Coffee Shop              12
Hotel                    10
Noodle House              7
Chinese Restaurant        6
Café                      4
Cha Chaan Teng            2
Japanese Restaurant       1
---------
                       name
2nd Most Common Venue      
Chinese Restaurant        8
Café                      7
Coffee Shop               6
Hotel                     6
Japanese Restaurant       6
Cha Chaan Teng            4
Fast Food Restaurant      2
Noodle House              2
Dim Sum Restaurant        1
---------
                       name
3rd Most Common Venue      
Coffee Shop               9
Cantonese Restaurant      6
Noodle House              6
Hotel                     4
Shopping Mall             4
Cha Chaan Teng            3
Japanese Restaurant       3
Chinese Restaurant        2
Thai Restaurant           2
Clothing Store            1
Hong Kong Restaurant      1
Park                      1
---------


In [33]:
query3=kr_merged.loc[kr_merged['Cluster Labels'] == 7, kr_merged.columns[[0] + list(range(15, kr_merged.shape[1]-7))]]
query3.head()

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
7,Chumchummi,7,Japanese Restaurant,Hotel,Dessert Shop
8,Banchan & Cook 반찬앤쿡 (Banchan & Cook),7,Japanese Restaurant,Hotel,Coffee Shop
13,Kobekyu Hong Kong,7,Japanese Restaurant,Hotel,Coffee Shop
16,Seoul House (首爾家),7,Japanese Restaurant,Coffee Shop,Cha Chaan Teng
19,Hon Wo Korean Restaurant (漢和韓國料理),7,Hotel,Japanese Restaurant,Coffee Shop


In [34]:
print(query3[['name','1st Most Common Venue']].groupby('1st Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query3[['name','2nd Most Common Venue']].groupby('2nd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')
print(query3[['name','3rd Most Common Venue']].groupby('3rd Most Common Venue').count().sort_values(by=['name'],ascending=False))
print('---------')

                       name
1st Most Common Venue      
Japanese Restaurant      17
Hotel                     8
Chinese Restaurant        2
---------
                       name
2nd Most Common Venue      
Hotel                    11
Japanese Restaurant       9
Coffee Shop               6
Cha Chaan Teng            1
---------
                       name
3rd Most Common Venue      
Coffee Shop              14
Noodle House              7
Café                      3
Dessert Shop              2
Cha Chaan Teng            1
---------


In [37]:
import matplotlib.cm as cm
import matplotlib.colors as colors

kr_merged=kr_merged.dropna(subset=['Cluster Labels'])
kr_merged['Cluster Labels']=kr_merged['Cluster Labels'].astype('int32')

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kr_merged['lat'], kr_merged['lng'], kr_merged['name'], kr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters